 # PyDub을 사용하여 음성 파일을 무음 구간을 기준으로 분리하고, 분리된 오디오 조각들을 최대 15초까지만 선택하여 WAV 파일로 저장하고, 마지막으로 무음을 제거하는 작업을 수행

In [ ]:
pip install pydub

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import subprocess


In [ ]:
# 입력 파일 경로와 출력 폴더를 설정

input_file = 'input_file.mp3'
output_folder = 'output_folder'

In [ ]:
# 입력 파일을 PyDub으로 로드하고, 샘플링 레이트, 채널, 샘플 넓이를 설정합니다.

sound = AudioSegment.from_file(input_file)
sound = sound.set_frame_rate(44100).set_channels(1).set_sample_width(2)

In [ ]:
# 이제, PyDub의 split_on_silence() 함수를 사용하여 입력된 음성 파일을 무음을 기준으로 자동으로 분리합니다.
# 이 함수는 최소 무음 길이, 무음으로 간주되는 dBFS 값, 분리된 각 음성 조각들 간의 추가적인 무음 길이를 인자로 받습니다.

audio_chunks = split_on_silence(sound,
    min_silence_len=1000, # 최소 무음 길이 (밀리초 단위)
    silence_thresh=-35, # 무음으로 간주되는 dBFS 값
    keep_silence=500 # 분리된 오디오 조각들 간의 추가적인 무음 길이 (밀리초 단위)
)

In [ ]:
# 분리된 음성 조각들 중 최대 15초까지만 선택하여 WAV 파일로 저장합니다. 
# 이 작업은 각 음성의 길이가 너무 긴 경우 파일 크기가 너무 커지는 것을 방지하기 위한 것

for i, chunk in enumerate(audio_chunks):
    if len(chunk) > 15000:
        chunk = chunk[:15000]
    output_file = os.path.join(output_folder, f'chunk_{i}.wav')
    chunk.export(output_file, format='wav')

In [ ]:
# 무음제거 코드

for filename in os.listdir(output_folder):
    if filename.endswith('.wav'):
        input_path = os.path.join(output_folder, filename)
        output_path = os.path.join(output_folder, f'processed_{filename}')
        subprocess.call(['ffmpeg', '-i', input_path, '-af', 'silenceremove=1:0:-50dB', '-y', output_path])
        os.remove(input_path)